In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cervical_Cancer"
cohort = "GSE138079"

# Input paths
in_trait_dir = "../../input/GEO/Cervical_Cancer"
in_cohort_dir = "../../input/GEO/Cervical_Cancer/GSE138079"

# Output paths
out_data_file = "../../output/preprocess/Cervical_Cancer/GSE138079.csv"
out_gene_data_file = "../../output/preprocess/Cervical_Cancer/gene_data/GSE138079.csv"
out_clinical_data_file = "../../output/preprocess/Cervical_Cancer/clinical_data/GSE138079.csv"
json_path = "../../output/preprocess/Cervical_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Identification of deregulated pathways, key regulators, and novel miRNA-mRNA interactions in HPV-mediated transformation. [mRNA cell lines-Agilent]"
!Series_summary	"Next to a persistent infection with high-risk human papillomavirus (HPV), molecular changes are required for the development of cervical cancer. To identify which molecular alterations drive carcinogenesis, we performed a comprehensive and longitudinal molecular characterization of HPV-transformed keratinocyte cell lines. Comparative genomic hybridization, mRNA, and miRNA expression analysis of four HPV-containing keratinocyte cell lines at eight different time points was performed. Data was analyzed using unsupervised hierarchical clustering, integrated longitudinal expression analysis, and pathway enrichment analysis. Biological relevance of identified key regulatory genes was evaluated in vitro and dual-luciferase assays were used to confirm predicted miRNA-mRNA interactions. We sh

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Dict, Any, Optional

# 1. Gene Expression Data Availability
# Based on the Series_title and Series_summary, this data contains mRNA microarray data from Agilent
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# For trait: The "transformation stage" in index 1 indicates different stages of cancer development
trait_row = 1

# For age: Not available in the sample characteristics
age_row = None

# For gender: Not explicitly available, but this is from male foreskin samples
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> int:
    """Convert transformation stage to binary: 1 for anchorage independent (more advanced cancer stage), 0 otherwise."""
    if isinstance(value, str):
        # Extract the value after colon
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        # Binary classification based on transformation stage
        if 'anchorage independent' in value.lower():
            return 1  # Advanced transformation stage
        elif 'immortal' in value.lower() or 'extended lifespan' in value.lower():
            return 0  # Earlier transformation stage
    return None

def convert_age(value: str) -> Optional[float]:
    """Convert age to continuous value."""
    # Not used as age is not available
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender to binary (0 for female, 1 for male)."""
    # Not used as gender is not explicitly available
    return None

# 3. Save Metadata
# trait_row is not None, meaning trait data is available
is_trait_available = trait_row is not None

# Initial filtering on usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Since this is a cell line study and not a human cohort study, and we don't have 
# access to the clinical_data variable from a previous step, we'll skip the clinical 
# feature extraction step for now. The trait information would need to be processed 
# when we have access to the actual data.


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, these appear to be numerical identifiers (12, 13, 14, etc.)
# rather than human gene symbols like BRCA1, TP53, etc.
# These are likely probe IDs or array-specific identifiers that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['266', '266', '266', '266', '266'], 'ROW': [170.0, 168.0, 166.0, 164.0, 162.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan], 'GO_ID': [nan, nan, nan, nan, nan], 'SEQUENCE': [nan, nan, nan, nan, nan], 'SPOT_ID.1': [nan, nan, nan, nan, nan], 'ORDER': [1.0, 2.0, 3.

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify relevant columns from the gene annotation data for mapping
# The gene identifiers in gene_data are numeric IDs
# From the preview, we need to map 'ID' in the annotation to a gene symbol column

# Looking at the columns in gene_annotation
print("Available columns in gene annotation:")
print(gene_annotation.columns.tolist())

# Let's get a better look at the annotation data beyond the first few rows
# to find rows that actually have gene symbol information
print("\nSample of gene annotation data with non-null gene symbols:")
sample_with_genes = gene_annotation[gene_annotation['GENE_SYMBOL'].notna()].head(5)
print(preview_df(sample_with_genes))

# 2. Create the gene mapping dataframe
# Based on the data, we'll map 'ID' to 'GENE_SYMBOL'
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'GENE_SYMBOL')
print("\nGene mapping dataframe:")
print(preview_df(mapping_df))

# 3. Apply the gene mapping to convert probe-level measurements to gene expression
gene_data_mapped = apply_gene_mapping(gene_data, mapping_df)

# Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_data_mapped)

# Preview the result
print("\nConverted gene expression data:")
print(preview_df(gene_data))


Available columns in gene annotation:
['ID', 'COL', 'ROW', 'NAME', 'SPOT_ID', 'CONTROL_TYPE', 'REFSEQ', 'GB_ACC', 'GENE', 'GENE_SYMBOL', 'GENE_NAME', 'UNIGENE_ID', 'ENSEMBL_ID', 'TIGR_ID', 'ACCESSION_STRING', 'CHROMOSOMAL_LOCATION', 'CYTOBAND', 'DESCRIPTION', 'GO_ID', 'SEQUENCE', 'SPOT_ID.1', 'ORDER']

Sample of gene annotation data with non-null gene symbols:
{'ID': ['12', '14', '15', '16', '18'], 'COL': ['266', '266', '266', '266', '266'], 'ROW': [148.0, 144.0, 142.0, 140.0, 136.0], 'NAME': ['A_24_P66027', 'A_23_P212522', 'A_24_P934473', 'A_24_P9671', 'A_24_P801451'], 'SPOT_ID': ['A_24_P66027', 'A_23_P212522', 'A_24_P934473', 'A_24_P9671', 'A_24_P801451'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_004900', 'NM_014616', nan, 'NM_001539', 'NM_006709'], 'GB_ACC': ['NM_004900', 'NM_014616', 'AK092846', 'NM_001539', 'NM_006709'], 'GENE': [9582.0, 23200.0, 100132006.0, 3301.0, 10919.0], 'GENE_SYMBOL': ['APOBEC3B', 'ATP11B', 'LOC100132006', 'DNAJA1', 'EHM


Converted gene expression data:
{'GSM4098797': [11.295818278999999, 4.511519006, 4.029339788, 7.998331606, 3.529187489], 'GSM4098798': [11.311701529, 4.350843151, 3.829114997, 6.938423613, 3.403435718], 'GSM4098799': [11.490747099, 3.940382921, 3.518664325, 8.071803433, 3.703306235], 'GSM4098800': [10.62815242, 4.278446209, 3.952369234, 7.445960991, 4.175610589], 'GSM4098801': [11.425336329, 4.211956199, 3.755241441, 7.389486419, 3.693382222], 'GSM4098802': [11.309095206, 3.807001272, 3.779241628, 8.267602906, 3.980138411], 'GSM4098803': [10.929429424, 3.925654364, 3.833418791, 8.355437797, 3.735606345], 'GSM4098804': [10.700006593000001, 4.314044971, 3.658411735, 7.448207267, 3.828141668], 'GSM4098805': [11.592457953, 3.520838128, 3.908003521, 7.832426589, 3.520838128], 'GSM4098806': [9.173432164000001, 4.137373263, 2.928105386, 8.132917185, 3.335657835], 'GSM4098807': [11.6466158, 4.51745926, 3.900382259, 8.405353632, 3.930806637], 'GSM4098808': [12.349148621, 4.781086546, 3.0854995

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Create clinical features directly from clinical_data using the conversion functions defined earlier
clinical_features_df = geo_select_clinical_features(
    clinical_data, 
    trait=trait, 
    trait_row=trait_row, 
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Now link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features_df, normalized_gene_data)
print("Linked data shape:", linked_data.shape)

# Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="This is an HPV-transformed keratinocyte cell line study focusing on transformation stages: 1 for anchorage independent (more advanced cancer stage), 0 for earlier stages."
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Cervical_Cancer/gene_data/GSE138079.csv
Clinical data saved to ../../output/preprocess/Cervical_Cancer/clinical_data/GSE138079.csv
Linked data shape: (64, 17902)


For the feature 'Cervical_Cancer', the least common label is '0.0' with 23 occurrences. This represents 35.94% of the dataset.
The distribution of the feature 'Cervical_Cancer' in this dataset is fine.



Linked data saved to ../../output/preprocess/Cervical_Cancer/GSE138079.csv
